In [1]:
!sudo apt install tesseract-ocr

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.


In [2]:
!pip install pytesseract

In [3]:
import pytesseract
import pprint
import shutil
import os
import random
try:
    from PIL import Image
except ImportError:
    import Image

In [4]:
from google.colab import files

uploaded = files.upload()

Saving receipt test.jpeg to receipt test.jpeg


In [ ]:
extractedInformation = pytesseract.image_to_string(Image.open('receipt example.jpg'))

In [ ]:
# Upload file
uploaded = files.upload()  # This is a dict

openai.api_key = "your-secret-api-key"

# Grab the first file from the upload
for filename in uploaded.keys():
    img = Image.open(filename)
    extractedInformation = pytesseract.image_to_string(img)

# Get the first uploaded file
for filename in uploaded:
    image = Image.open(io.BytesIO(uploaded[filename]))
    extractedInformation = pytesseract.image_to_string(image)


def parse_receipt(receipt_text):
    # Split by newline and strip whitespace
    lines = [line.strip() for line in receipt_text.strip().split('\n') if line.strip()]
    return lines

import openai

def interpret_details(parsed_text):
    prompt = (
        "You are an expert on reading receipts. "
        "Please classify the following receipt text into a JSON-style dictionary with these fields:\n\n"
        "1. 'store_name': the name of the store (or 'unknown' if not found)\n"
        "2. 'address': the address of the store (if known, else 'unknown')\n"
        "3. 'items': a dictionary of purchased items where the key is the item name and the value is the price\n"
        "4. 'total': the total amount spent\n\n"
        "Receipt text:\n"
        f"{parsed_text}"
    )

    response = openai.ChatCompletion.create(
        model="gpt-4",  # or "gpt-3.5-turbo" depending on what you're using
        messages=[
            {"role": "system", "content": "You are an assistant that extracts structured data from receipts."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.3
    )

    # Return the model's response
    return response['choices'][0]['message']['content']




Saving receipt example.jpg to receipt example (3).jpg


In [9]:
extractedInformation

'Main Street Restaurant\n6332 Business Drive\nSuite 528\nPalo Alto California 94301\n575-1628095\n\nFri 04/07/2017 11:36 AM\n\nMerchant ID: 9hqjxvufdr\nTerminal ID: eet ia\n\nTransaction ID: #e6d598ef\nType: CREDIT\n\nPURCHASE\nNumber : AXXXXXXXXXXXOO4 1\nEntry Mode: Swiped\nCard Type: DISCOVER\n\nResponse: APPROVED\nApproval Code: 819543\n\nSub Total USD$ 25.23\ncls\n\nUSD$ 29.01\n\nThanks for supporting\nlocal business!\n\nTHANK YOU\n\n \n\x0c'

In [ ]:
import streamlit as st
from PIL import Image
import pytesseract
import datetime
from io import BytesIO
from openai import OpenAI
import os

# Setup OpenAI (make sure to set your API key securely)
openai_api_key = os.getenv("OPENAI_API_KEY")

# Function to extract receipt data using GPT

def interpret_details(parsed_text):
    prompt = (
        "You are an expert on reading receipts. "
        "Please classify the following receipt text into a JSON-style dictionary with these fields:\n"
        "1. 'store_name'\n"
        "2. 'address'\n"
        "3. 'items': dictionary of items and prices\n"
        "4. 'total': total amount\n\n"
        f"Receipt text:\n{parsed_text}"
    )

    client = OpenAI(api_key=openai_api_key)

    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are an assistant that extracts structured data from receipts."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.2
    )

    return response.choices[0].message.content

# Streamlit UI
st.title("📸 Receipt Parser and Logger")

uploaded_file = st.file_uploader("Upload a receipt image", type=["png", "jpg", "jpeg"])
captured_image = st.camera_input("Or take a picture")

if uploaded_file or captured_image:
    image_file = uploaded_file if uploaded_file else captured_image
    image = Image.open(image_file)
    st.image(image, caption="Uploaded Receipt", use_column_width=True)

    with st.spinner("Extracting text..."):
        parsed_text = pytesseract.image_to_string(image)
        try:
            model_output = interpret_details(parsed_text)
            st.success("Receipt data extracted successfully!")
        except Exception as e:
            st.error(f"Failed to call model: {e}")
            st.stop()

    st.subheader("🧾 Receipt Details")

    person = st.text_input("Person (label)")
    category = st.selectbox("Category of Purchase", ["Travel", "Meals", "Office Supplies", "Miscellaneous"])

    # Try parsing the output
    import json
    try:
        data = json.loads(model_output)
        total = st.number_input("Total Amount", value=float(data.get("total", 0.0)))
    except:
        st.warning("Could not parse structured receipt data. You can enter manually.")
        data = {"items": {}, "total": 0.0}
        total = st.number_input("Total Amount", value=0.0)

    purchase_date = st.date_input("Date of Purchase", datetime.date.today())
    effective_date = st.date_input("Effective Date", datetime.date.today())

    st.markdown("---")
    st.subheader("🛒 Itemisation")

    # Create a session state to track items
    if "items" not in st.session_state:
        st.session_state.items = list(data.get("items", {}).items())

    # Buttons to add or remove items
    if st.button("➕ Add Item"):
        st.session_state.items.append(("", 0.0))

    if st.button("➖ Remove Last Item") and st.session_state.items:
        st.session_state.items.pop()

    updated_items = []
    for i, (item, price) in enumerate(st.session_state.items):
        col1, col2 = st.columns([3, 1])
        item_name = col1.text_input(f"Item {i+1}", value=item, key=f"item_{i}")
        item_price = col2.number_input("$", value=float(price), key=f"price_{i}")
        updated_items.append((item_name, item_price))

    st.session_state.items = updated_items

    st.markdown("---")

    if st.button("✅ Submit"):
        submission_data = {
            "person": person,
            "category": category,
            "total": total,
            "purchase_date": str(purchase_date),
            "effective_date": str(effective_date),
            "items": dict(updated_items)
        }

        st.success("Submission Successful!")
        st.json(submission_data)
        # TODO: send to backend/database as needed

In [8]:
extractedInformation

'Main Street Restaurant\n6332 Business Drive\nSuite 528\nPalo Alto California 94301\n575-1628095\n\nFri 04/07/2017 11:36 AM\n\nMerchant ID: 9hqjxvufdr\nTerminal ID: eet ia\n\nTransaction ID: #e6d598ef\nType: CREDIT\n\nPURCHASE\nNumber : AXXXXXXXXXXXOO4 1\nEntry Mode: Swiped\nCard Type: DISCOVER\n\nResponse: APPROVED\nApproval Code: 819543\n\nSub Total USD$ 25.23\ncls\n\nUSD$ 29.01\n\nThanks for supporting\nlocal business!\n\nTHANK YOU\n\n \n\x0c'

In [ ]:
pprint.pp(extractedInformation)

('  \n'
 '\n'
 'See back of receipt for your chance\n'
 'to win $1000\n'
 'ID #: — 7L2TOWIM25F\n'
 '\n'
 '   \n'
 '\n'
 '     \n'
 '  \n'
 '\n'
 'Walmart >/<.\n'
 '\n'
 'Save money. Live better.\n'
 '\n'
 '( 701 ) 223 - 3066\n'
 'MANAGER TIM MORRIS\n'
 '2717 ROCK ISLAND PL\n'
 'BISMARCK ND 58504\n'
 'ST# 01534 OP# 009048 TE# 48 TRH 04455\n'
 '\n'
 '       \n'
 '      \n'
 '\n'
 '6 WING PLATE 020108870398 3.98 P\n'
 'ASST 27 063099656595 4.88 X\n'
 'CUTIE CAR 063099656644 12.88 X\n'
 '\n'
 'SUBTOTAL 21.74\n'
 '\n'
 'TAX 1 6.500 % 1.15\n'
 '\n'
 'TAX 4 7.500 % 0.30\n'
 '\n'
 'TOTAL 23.19\n'
 'DEBIT TEND 23.19\n'
 'CHANGE DUE 0.00\n'
 '\n'
 ' \n'
 '\n'
 'EFT DEBIT PAY FROM PRIMARY\n'
 '23.19 TOTAL PURCHASE\n'
 'US DEBIT xxKK xxKK xxx 2696 I 0\n'
 '\n'
 'REF # 731700494718\n'
 'NETWORK ID. 0081 APPR CODE 936886\n'
 '\n'
 'US DEBIT\n'
 '\n'
 'AID A0Q000000980840\n'
 '\n'
 'TC 58B77239DA5B3B44\n'
 '*NO SIGNATURE REQUIRED\n'
 'TERMINAL # SCO11090\n'
 '\n'
 '11/13/17 12:34:04\n'
 '# ITEMS SOLD

In [ ]:
def parse_receipt(receipt_text):
    # Split by newline and strip whitespace
    lines = [line.strip() for line in receipt_text.strip().split('\n') if line.strip()]
    return lines

parse_receipt(extractedInformation)

['See back of receipt for your chance',
 'to win $1000',
 'ID #: — 7L2TOWIM25F',
 'Walmart >/<.',
 'Save money. Live better.',
 '( 701 ) 223 - 3066',
 'MANAGER TIM MORRIS',
 '2717 ROCK ISLAND PL',
 'BISMARCK ND 58504',
 'ST# 01534 OP# 009048 TE# 48 TRH 04455',
 '6 WING PLATE 020108870398 3.98 P',
 'ASST 27 063099656595 4.88 X',
 'CUTIE CAR 063099656644 12.88 X',
 'SUBTOTAL 21.74',
 'TAX 1 6.500 % 1.15',
 'TAX 4 7.500 % 0.30',
 'TOTAL 23.19',
 'DEBIT TEND 23.19',
 'CHANGE DUE 0.00',
 'EFT DEBIT PAY FROM PRIMARY',
 '23.19 TOTAL PURCHASE',
 'US DEBIT xxKK xxKK xxx 2696 I 0',
 'REF # 731700494718',
 'NETWORK ID. 0081 APPR CODE 936886',
 'US DEBIT',
 'AID A0Q000000980840',
 'TC 58B77239DA5B3B44',
 '*NO SIGNATURE REQUIRED',
 'TERMINAL # SCO11090',
 '11/13/17 12:34:04',
 '# ITEMS SOLD 3',
 'TC# 2188 7274 6444 1567 3762',
 'NUL',
 'ii Low Prices You Can Trust. Every Day.',
 ': 11/13/17 12:34:11',
 'Store receipts on your phone. Walmart P',
 '= |']

In [ ]:
import cv2
import pytesseract
import numpy as np
from io import BytesIO
from PIL import Image

In [ ]:
def image_to_text(image_bytes):
    # Convert bytes to an image
    image = Image.open(BytesIO(image_bytes))

    # Convert image to OpenCV format
    image = np.array(image)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    # Convert to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply thresholding to improve OCR accuracy
    _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Perform OCR
    text = pytesseract.image_to_string(thresh, config='--psm 6')

    return text
extractedInformation

In [ ]:
# Prompt user for file name
extractedInformation
try:
    with open(file_name, 'rb') as f:
        image_bytes = f.read()
    extracted_text = image_to_text(image_bytes)
    print("Extracted Text:\n", extracted_text)
except FileNotFoundError:
    print("Error: File not found. Please check the file name and try again.")

KeyboardInterrupt: Interrupted by user